In [223]:
import arcpy
import numpy as np
# %matplotlib inline

import matplotlib.pyplot as plt

In [165]:
arcpy.env.addOutputsToMap = True
arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'Default.gdb'

In [169]:
import pandas as pd
# from arcgis.features.use_proximity import create_buffers
from arcgis.features import SpatialDataFrame
from arcgis.features import GeoAccessor
from arcgis.learn import prepare_tabulardata

In [203]:
factors = np.array(['intersection','market','hotel','bank','education','temple','office','condo','house','factory','Cafe'])
bufferLayer = 'buffers'
inputData = 'samplepoint'

In [204]:
trainLayer = GeoAccessor.from_featureclass(inputData)
buffer = arcpy.analysis.Buffer(inputData,bufferLayer,'500 Meters')

In [205]:
arcpy.env.addOutputsToMap = False
for f in factors:
    print(f)
    arcpy.analysis.SummarizeWithin(bufferLayer,f,'count')
    countDf = GeoAccessor.from_featureclass('count')
    trainLayer[f + '_count'] = countDf['Point_Count']

intersection
market
hotel
bank
education
temple
office
condo
house
factory
Cafe


In [206]:
arcpy.analysis.SummarizeWithin(bufferLayer,'l_trans','count')
countDf = GeoAccessor.from_featureclass('count')
trainLayer['road_Length'] = countDf['SUM_Length_KILOMETERS']


In [231]:
trainLayer.hist()
plt.show()

In [208]:
countDf = GeoAccessor.from_featureclass('count')
countDf['SUM_Length_KILOMETERS']
trainLayer['road_Length'] = countDf['SUM_Length_KILOMETERS']

In [210]:
list(trainLayer.columns[5:])

['intersection_count', 'market_count', 'hotel_count', 'bank_count', 'education_count', 'temple_count', 'office_count', 'condo_count', 'house_count', 'factory_count', 'Cafe_count', 'road_Length']

In [211]:
tempLayer = r'in_memory\temp'
trainLayer.spatial.to_featureclass(tempLayer)

'in_memory\\temp'

In [212]:
for f in factors:
    print(f)
    result = arcpy.Near_analysis(tempLayer,f)
    temp = GeoAccessor.from_featureclass(tempLayer,fields=['NEAR_DIST'])
    trainLayer[f + '_Dist'] = temp['NEAR_DIST']
trainLayer

intersection
market
hotel
bank
education
temple
office
condo
house
factory
Cafe


,objectid,field1,field2,field3,SHAPE,intersection_count,market_count,hotel_count,bank_count,education_count,temple_count,office_count,condo_count,house_count,factory_count,cafe_count,road_length,intersection_Dist,market_Dist,hotel_Dist,bank_Dist,education_Dist,temple_Dist,office_Dist,condo_Dist,house_Dist,factory_Dist,Cafe_Dist
0,1,1.118930e+07,1.539893e+06,2,"{""x"": 11189690.370099999, ""y"": 1539601.1882000...",0,0,0,3,5,0,2,2,1,35,6,1.541712,1118.494330,1059.622487,547.553322,377.138868,401.932600,665.382850,877.649827,193.132814,121.475127,21.628051,181.496923
1,2,1.118678e+07,1.540614e+06,2,"{""x"": 11186530.568799999, ""y"": 1540813.3722000...",0,1,1,0,3,1,0,2,1,19,11,3.395373,672.321863,437.944532,431.706343,378.473144,235.038586,185.520142,162.216110,228.802601,354.908952,200.005863,191.560706
2,3,1.119064e+07,1.543173e+06,3,"{""x"": 11190549.4214, ""y"": 1543297.8480000012, ...",0,0,0,0,0,0,0,0,0,0,0,0.000000,218.874787,363.467077,174.632926,176.826180,116.652967,351.581617,101.802883,170.103285,1659.840665,64.397973,116.270144
3,4,1.119056e+07,1.540567e+06,2,"{""x"": 11190643.7271, ""y"": 1540457.575100001, ""...",0,0,2,2,1,0,0,4,8,19,23,3.300709,160.671889,46.804714,708.903728,113.036041,77.766739,645.486647,575.553728,247.315077,585.023888,51.544070,70.657822
4,5,1.118731e+07,1.539573e+06,3,"{""x"": 11186886.963, ""y"": 1539250.8159999996, ""...",0,0,0,0,0,0,0,0,1,22,11,0.808671,462.551267,233.688166,841.835579,139.762642,590.639835,550.955535,190.009279,58.603421,507.768555,188.129194,68.857405
5,6,1.119684e+07,1.543749e+06,3,"{""x"": 11197244.405000001, ""y"": 1543637.3630999...",0,0,3,1,4,3,2,4,1,2,23,1.307601,282.497178,868.623478,71.947260,186.048529,220.600333,919.362862,220.600333,214.270593,362.635291,341.570043,92.067682
6,7,1.119284e+07,1.544861e+06,2,"{""x"": 11192977.314399999, ""y"": 1544664.9820000...",7,2,43,18,8,1,35,11,0,8,166,7.770642,140.256199,966.430611,56.555529,56.555529,49.384513,790.031869,56.555529,153.222814,232.313598,524.904338,32.273112
7,8,1.118814e+07,1.542284e+06,3,"{""x"": 11187958.355999999, ""y"": 1542139.5976000...",1,1,0,0,2,0,0,3,3,1,12,1.194740,583.108377,570.373408,232.387246,342.717834,332.394631,520.874421,159.913454,46.552437,369.598895,163.620886,152.510305
8,9,1.119024e+07,1.542996e+06,2,"{""x"": 11190252.4016, ""y"": 1542907.397500001, ""...",0,2,1,1,7,1,0,5,0,5,16,1.731343,166.223319,240.617025,74.112358,229.748842,131.203314,701.815016,83.133389,213.061235,1722.080215,92.266407,72.673760
9,10,1.118826e+07,1.538536e+06,4,"{""x"": 11188724.498300001, ""y"": 1538459.4763000...",1,1,7,8,7,0,1,9,2,4,38,6.931012,185.727141,992.677776,264.852587,437.062487,217.775206,226.672615,348.805994,51.933005,451.699331,264.666321,128.451357


In [213]:
result = arcpy.Near_analysis(tempLayer,'l_trans')
temp = GeoAccessor.from_featureclass(tempLayer,fields=['NEAR_DIST'])
trainLayer['road_Dist'] = temp['NEAR_DIST']

In [197]:
trainDf = prepare_tabulardata(trainLayer,'class',list(trainLayer.columns[5:]),val_split_pct=0.1,batch_size=4)
trainDf.show_batch()

,Cafe_Dist,bank_Dist,bank_count,cafe_count,class,condo_Dist,condo_count,education_Dist,education_count,factory_Dist,factory_count,hotel_Dist,hotel_count,house_Dist,house_count,intersection_Dist,intersection_count,market_Dist,market_count,office_Dist,office_count,road_Dist,road_length,temple_Dist,temple_count
4,260.826542,1026.361004,0,11,1,1518.614629,0,915.727064,0,60.237898,22,1923.229054,0,460.019315,1,5193.372027,0,1195.581947,0,526.380438,0,276.906844,0.808671,2207.168953,0
5,34.437106,266.014067,1,23,2,92.828253,4,247.158984,4,207.375968,2,259.447493,3,372.421610,1,631.411836,0,597.437488,0,356.021246,2,55.143523,1.307601,281.974167,3
9,97.835211,100.777829,8,38,3,119.129801,9,211.818905,7,177.361823,4,113.607477,7,417.914194,2,244.297271,1,145.365380,1,252.198733,1,52.312563,6.931012,579.517813,0
24,307.660378,625.845031,0,2,4,556.088527,0,669.610100,0,214.640651,2,416.626296,1,264.137924,6,958.114509,0,860.767326,0,809.562065,0,108.129240,1.020655,1020.072863,0
26,69.521161,148.660262,8,59,3,74.044533,20,331.473498,3,467.606902,1,144.863115,12,579.315357,0,570.435402,0,512.226438,0,153.944879,9,83.471723,2.665078,492.502957,1


In [198]:
from arcgis.learn import FullyConnectedNetwork
ANNModel = FullyConnectedNetwork(trainDf)

In [199]:
lr = ANNModel.lr_find()

In [200]:
ANNModel.fit(epochs=500,lr=lr)

epoch     train_loss  valid_loss  time    
0         1.428770    1.345911    00:00     
1         1.396497    1.348003    00:00     
2         1.390197    1.349988    00:00     
3         1.383754    1.352326    00:00     
4         1.376048    1.353936    00:00     
5         1.368469    1.354719    00:00     
6         1.365581    1.356970    00:00     
7         1.353853    1.357193    00:00     
8         1.345845    1.359125    00:00     
9         1.337302    1.360634    00:00     
10        1.327486    1.362974    00:00     
11        1.318817    1.366787    00:00     
12        1.309426    1.368468    00:00     
13        1.296330    1.366943    00:00     
14        1.289532    1.369461    00:00     
15        1.276748    1.374667    00:00     
16        1.267807    1.374476    00:00     
17        1.252907    1.375636    00:00     
18        1.235997    1.377774    00:00     
19        1.221138    1.378115    00:00     
20        1.207469    1.377525    00:00     
21        1.

181       0.233891    1.965873    00:00     
182       0.248713    2.220741    00:00     
183       0.268599    1.963784    00:00     
184       0.260502    1.648317    00:00     
185       0.259070    1.780335    00:00     
186       0.252829    1.615147    00:00     
187       0.237952    1.763622    00:00     
188       0.249716    0.930439    00:00     
189       0.258704    2.058302    00:00     
190       0.244197    2.338339    00:00     
191       0.238685    1.862313    00:00     
192       0.221828    1.312971    00:00     
193       0.210952    1.203879    00:00     
194       0.206727    1.724236    00:00     
195       0.199006    2.000283    00:00     
196       0.202114    1.487567    00:00     
197       0.225331    1.498932    00:00     
198       0.238801    1.227400    00:00     
199       0.232640    1.778851    00:00     
200       0.237659    1.684105    00:00     
201       0.257183    1.785650    00:00     
202       0.242036    1.551046    00:00     
203       

363       0.066093    1.377565    00:00     
364       0.063283    1.279422    00:00     
365       0.073740    1.549785    00:00     
366       0.072136    1.264640    00:00     
367       0.086761    1.142715    00:00     
368       0.078119    0.993907    00:00     
369       0.082348    1.008787    00:00     
370       0.075924    0.992364    00:00     
371       0.081804    1.309104    00:00     
372       0.084086    1.324076    00:00     
373       0.081759    1.317326    00:00     
374       0.074584    1.236778    00:00     
375       0.072153    1.179441    00:00     
376       0.066967    1.059366    00:00     
377       0.061623    1.214007    00:00     
378       0.074305    1.338428    00:00     
379       0.067974    1.870152    00:00     
380       0.066792    2.138858    00:00     
381       0.063766    1.949791    00:00     
382       0.066881    1.958225    00:00     
383       0.080712    1.729353    00:00     
384       0.075403    1.766865    00:00     
385       

In [202]:
ANNModel.show_results(30)
# ANNModel.score()

,Cafe_Dist,bank_Dist,bank_count,cafe_count,class,condo_Dist,condo_count,education_Dist,education_count,factory_Dist,factory_count,hotel_Dist,hotel_count,house_Dist,house_count,intersection_Dist,intersection_count,market_Dist,market_count,office_Dist,office_count,road_Dist,road_length,temple_Dist,temple_count,prediction_results
0,87.681033,608.449975,0,12,1,73.396215,3,170.318502,2,255.932983,1,874.144097,0,370.532438,3,498.440793,1,406.726427,1,546.081106,0,69.329869,1.194740,880.958283,0,1
1,319.210934,586.421240,0,11,1,120.990189,2,292.762605,3,265.496789,19,299.671686,1,475.083189,1,2443.996548,0,334.136960,1,770.820349,0,310.526540,3.395373,404.916891,1,1
2,541.972834,761.204143,0,0,1,1736.749648,0,1125.694612,0,1422.848377,0,3007.949306,0,188.758143,3,817.348546,0,465.259030,1,3920.826562,0,319.410552,0.735503,1260.772687,0,1


In [215]:
predictedDf = ANNModel.predict(trainLayer[list(trainLayer.columns[5:])],prediction_type='dataframe')

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\learn\models\_layer_learner.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["prediction_results"] = self._df_predict(processed_dataframe.copy())


In [239]:
predictedLayer = GeoAccessor.from_featureclass('samplepoint')
predictedLayer['predicted'] = predictedDf['prediction_results']
predictedLayer

,OBJECTID,Field1,Field2,Field3,SHAPE,predicted
0,1,1.118930e+07,1.539893e+06,2,"{""x"": 11189690.370099999, ""y"": 1539601.1882000...",1
1,2,1.118678e+07,1.540614e+06,2,"{""x"": 11186530.568799999, ""y"": 1540813.3722000...",1
2,3,1.119064e+07,1.543173e+06,3,"{""x"": 11190549.4214, ""y"": 1543297.8480000012, ...",4
3,4,1.119056e+07,1.540567e+06,2,"{""x"": 11190643.7271, ""y"": 1540457.575100001, ""...",1
4,5,1.118731e+07,1.539573e+06,3,"{""x"": 11186886.963, ""y"": 1539250.8159999996, ""...",1
5,6,1.119684e+07,1.543749e+06,3,"{""x"": 11197244.405000001, ""y"": 1543637.3630999...",2
6,7,1.119284e+07,1.544861e+06,2,"{""x"": 11192977.314399999, ""y"": 1544664.9820000...",2
7,8,1.118814e+07,1.542284e+06,3,"{""x"": 11187958.355999999, ""y"": 1542139.5976000...",1
8,9,1.119024e+07,1.542996e+06,2,"{""x"": 11190252.4016, ""y"": 1542907.397500001, ""...",1
9,10,1.118826e+07,1.538536e+06,4,"{""x"": 11188724.498300001, ""y"": 1538459.4763000...",3


In [242]:
predictedLayer.spatial.to_featureclass('Default.gdb/predicted')

'D:\\Student\\AGP\\arcgis-notebooks-tutorial\\notebook_basics\\Default.gdb\\predicted'

In [241]:
arcpy.env.workspace

'Default.gdb'

In [136]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
standard1 = StandardScaler()



In [142]:
trainNorm = standard1.fit_transform(trainLayer[list(trainLayer.columns[5:])])

In [155]:
df = pd.DataFrame(trainNorm,columns=list(trainLayer.columns[5:]))
normDf = pd.concat([trainLayer['class'],df],axis=1)
normDf

,class,hotel_count,hotel_Dist,l_trans_Dist,road_Length
0,1,-0.562299,-0.171612,-0.202102,-0.654966
1,1,-0.488076,-0.150347,-0.134861,0.242431
2,4,-0.562299,5.628136,5.630165,-1.401342
3,2,-0.413852,-0.221642,-0.213616,0.196603
4,1,-0.562299,-0.166157,-0.144797,-1.009846
5,2,-0.339629,-0.227462,-0.210332,-0.768304
6,3,2.629311,-0.215228,-0.211377,2.360593
7,1,-0.562299,-0.213044,-0.206140,-0.822942
8,1,-0.488076,-0.179364,-0.160280,-0.563161
9,3,-0.042735,-0.210294,-0.211169,1.954110


In [152]:
normDf[list(normDf.columns[1:])]

,0,1,2,3
0,-0.562299,-0.171612,-0.202102,-0.654966
1,-0.488076,-0.150347,-0.134861,0.242431
2,-0.562299,5.628136,5.630165,-1.401342
3,-0.413852,-0.221642,-0.213616,0.196603
4,-0.562299,-0.166157,-0.144797,-1.009846
5,-0.339629,-0.227462,-0.210332,-0.768304
6,2.629311,-0.215228,-0.211377,2.360593
7,-0.562299,-0.213044,-0.206140,-0.822942
8,-0.488076,-0.179364,-0.160280,-0.563161
9,-0.042735,-0.210294,-0.211169,1.954110


In [156]:
trainDf = prepare_tabulardata(normDf,'class',list(normDf.columns[1:]),val_split_pct=0.2,batch_size=4)
trainDf.show_batch()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\arcgis\learn\_utils\tabular_data.py:876: UserWarning: Dataframe is not spatial, Rasters and distance layers will not work
  warnings.warn("Dataframe is not spatial, Rasters and distance layers will not work")


,class,hotel_Dist,hotel_count,l_trans_Dist,road_Length
3,2,-0.221642,-0.413852,-0.213616,0.196603
5,2,-0.227462,-0.339629,-0.210332,-0.768304
21,4,-0.179141,-0.488076,-0.190095,-0.436251
26,3,-0.217961,0.328383,-0.201960,-0.111120
29,3,-0.205743,-0.562299,-0.211416,0.539542


In [157]:
from arcgis.learn import MLModel
DTree = MLModel(trainDf,'sklearn.tree.DecisionTreeClassifier')

In [158]:
DTree.fit()
DTree.show_results(30)

,class,hotel_Dist,hotel_count,l_trans_Dist,road_Length,class_results
1,1,-0.150347,-0.488076,-0.134861,0.242431,2
4,1,-0.166157,-0.562299,-0.144797,-1.009846,4
7,1,-0.213044,-0.562299,-0.206140,-0.822942,4
8,1,-0.179364,-0.488076,-0.160280,-0.563161,2
23,4,-0.153605,-0.562299,-0.191891,-0.052380,2
28,1,-0.179239,-0.413852,-0.155583,-0.211394,1
32,3,-0.205591,-0.339629,-0.159329,-0.985129,1
